In [1]:
import torch
import torchtext
from torchtext import data, datasets
from transformers import AdamW, BertConfig, BertTokenizer, BertForTokenClassification

I1224 01:09:57.401601 139955959142208 file_utils.py:35] PyTorch version 1.0.0 available.
W1224 01:09:58.151705 139955959142208 __init__.py:28] To use data.metrics please install scikit-learn. See https://scikit-learn.org/stable/index.html


In [6]:
bert_model_type = 'bert-base-uncased'

In [2]:
TEXT = data.Field(sequential=True, lower=True,
                  tokenize=lambda x: x.split(','),
                  preprocessing=lambda x: ['[CLS]'] + x[1:-1] + ['[SEP]'])
SLOTS = data.Field(sequential=True, lower=True,tokenize=lambda x: x.split(','))

In [3]:
class JointSlotIntent(data.Dataset):
    def __init__(self, slot_field):
        self.__slots = data.TabularDataset("data/atis.slots.train.csv", format='tsv',
                                            fields=[('slots', slot_field)])
        __intents = data.TabularDataset("data/atis.intent.train.csv", format='tsv',
                                              fields=[('intent', data.Field(lower=True))])
        for intent, slot in zip(__intents, self.__slots):
            slot.slots[0] = intent.intent[0]
        del __intents
        slot_field.build_vocab(self.__slots)
            
    def __len__(self):
        return len(self.__slots)
    
    def __getitem__(self, idx):        
        return self.__slots[idx]

In [4]:
sents = data.TabularDataset("data/atis.sentences.train.csv", format="tsv",
                            fields=[('sent', TEXT)])
TEXT.build_vocab(sents)
joint = JointSlotIntent(SLOTS)

In [11]:
tokenizer = BertTokenizer.from_pretrained(bert_model_type, do_basic_tokenize=False)

I1224 01:16:51.775768 139955959142208 tokenization_utils.py:398] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/sduddu/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [24]:
x = [tokenizer.wordpiece_tokenizer.tokenize(x) for x in sents[0].sent]
y = tokenizer.convert_tokens_to_ids(sents[0].sent)
print ("\t", len(x), x)
print ("\t", len(y), y)
print ("\t", tokenizer.tokenize(' '.join(sents[0].sent)))

	 20 [['[CLS]'], ['i'], ['want'], ['to'], ['fly'], ['from'], ['boston'], ['at'], ['83', '##8'], ['am'], ['and'], ['arrive'], ['in'], ['denver'], ['at'], ['111', '##0'], ['in'], ['the'], ['morning'], ['[SEP]']]
	 20 [101, 1045, 2215, 2000, 4875, 2013, 3731, 2012, 100, 2572, 1998, 7180, 1999, 7573, 2012, 100, 1999, 1996, 2851, 102]
	 ['[CLS]', 'i', 'want', 'to', 'fly', 'from', 'boston', 'at', '83', '##8', 'am', 'and', 'arrive', 'in', 'denver', 'at', '111', '##0', 'in', 'the', 'morning', '[SEP]']
